In [9]:
%pip install python-dotenv

from dotenv import load_dotenv
import os
import dspy

gemini_api_key = os.getenv("GEMINI_API_KEY")

print("Loaded:", gemini_api_key[:5] + "..." if gemini_api_key else "Not found")

lm = dspy.LM("gemini/gemini-2.5-flash", api_key=gemini_api_key)
dspy.configure(lm=lm)

qa = dspy.Predict('question: str -> response: str')
response = qa(question = "quantos presidentes o brasil ja teve?")

# print(f'{response.response}')

Note: you may need to restart the kernel to use updated packages.
Loaded: Not found


In [ ]:
dspy.inspect_history(n = 1)

In [ ]:

import mlflow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("DSPy")

mlflow.dspy.autolog()

In [ ]:
import requests
import os

url = "https://huggingface.co/dspy/cache/resolve/main/ragqa_arena_tech_corpus.jsonl"
output_path = "./data/ragqa_arena_tech_corpus.jsonl"

os.makedirs(os.path.dirname(output_path), exist_ok=True)

response = requests.get(url)

with open(output_path, "wb") as f:
    f.write(response.content)

print(f"✅ File downloaded to: {output_path}")


In [3]:
# usando Embeddings e fazendo busca top-K localmente

# %pip install -U faiss-cpu
# %pip install sentence_transformers

import json
import dspy
from sentence_transformers import SentenceTransformer

max_characters = 6000 # para truncar mais de 99% dos documentos
topk_docs_to_retrieve = 5 # numero de documentos para recuperar por query de busca

with open("./data/ragqa_arena_tech_corpus.jsonl") as f:
    corpus = [json.loads(line)['text'][:max_characters] for line in f]
    print(f'Loaded {len(corpus)} documents. Will encode them below.')

model = SentenceTransformer("sentence-transformers/static-retrieval-mrl-en-v1", device="cpu")

embedder = dspy.Embedder(model.encode)
search = dspy.retrievers.Embeddings(embedder=embedder, corpus=corpus, k=topk_docs_to_retrieve)



Loaded 28436 documents. Will encode them below.
Training a 32-byte FAISS index with 337 partitions, based on 28436 x 1024-dim embeddings


In [8]:
class RAG(dspy.Module):
    def __init__(self):
        self.respond = dspy.Predict('context, question -> response')
    
    def forward(self, question):
        context = search(question).passages
        return self.respond(context=context, question=question)

rag = RAG()
rag(question="what are high memory and low memory on linux?")

Prediction(
    response='In the context of memory, Low Memory is used for the kernel, while High Memory is designated for application space. This separation ensures that user-space applications cannot access kernel-space memory.'
)

c:\Users\andre\miniconda3\envs\light-rag\Lib\site-packages\pydantic\main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 7: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## re...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


In [ ]:
from dspy.evaluate import SemanticF1

metric = SemanticF1(decompositional=True)

# pred = cot(**example.inputs())
# Avalia a RAG

evaluate(RAG()) # NAO FUNCIONA!

NameError: name 'cot' is not defined